# Lag STAC fra GeoTiff

### Input: 
Mappe med .tif og .tfw -filer. Filer kan lastes ned fra Geonorge her: [(N50 Raster (UTM32) - Rutevis)](https://kartkatalog.geonorge.no/metadata/n50-raster-utm32-rutevis/bbe517ba-3b3a-4c96-a78e-104f6c4804d1?search=n50)

### Output:
**/stac_catalog** som inneholder *catalog.json* (oversikt) og egen mappe for hvert innleste kartblad.

## Import av bibliotek 

In [1]:
import os
import rasterio
import utils
import shutil
import subprocess
from pathlib import Path
from pystac import Catalog, Item, Asset
from datetime import datetime
from shapely.geometry import box, mapping

## Mappestruktur
Her er det tenkt at koden kjøres i en mappe som har underkatalogene "Input" (inneholder original Tiff-filene) og "Output" (mål for STAC-katalog)

N50TilSTAC/  
├─ Input/  
└─ Output/

Det vil også bli en "COG"-mappe:
N50TilSTAC/COG

In [2]:
# Definer input/output mapper
inputFolder = utils.get_workdir() / "Input"
outputFolder = utils.get_workdir() / "Output"

# Sjekk om COG-mappe eksisterer
COGFolder = utils.get_workdir() / "COG"
if COGFolder.exists():
    for item in COGFolder.iterdir():
        if item.is_file():
            item.unlink()
        elif item.is_dir():
            shutil.rmtree(item)
else:
    # Opprett COG-mappe hvis den ikke finnes
    COGFolder.mkdir(parents=True, exist_ok=True)

catalog = Catalog(id="N50-test-catalog", description="STAC Catalog for N50 maps")

## UDF: Konverter GeoTiff til COG

Funksjonen konverterer alle .tif-filer i Input-mappa til COG filer.  
Resultatet (COG-filene) lagres i "N50TilSTAS/COG/".

In [ ]:
def convert_folder_to_cog(input_folder, COGFolder):
    input_folder = Path(input_folder)
    COGFolder = Path(COGFolder)
    COGFolder.mkdir(parents=True, exist_ok=True)

    # Tellere
    nTellerAntTiffFiler = 0 # Antall TIFF-filer
    nTellerAntBehandla = 0 # Antall behandlede filer

    # Gå gjennom alle .tif-filer i input-mappen
    for tif_file in input_folder.glob("*.tif"):
        output_file = COGFolder / tif_file.name  # Same name
        print(f"Konverterer {tif_file.name} til COG...")
        # Tell antall filer som skal behandles
        nTellerAntTiffFiler += 1

        cmd = [
            "gdal_translate", str(tif_file), str(output_file),
            "-of", "COG",
            "-co", "COMPRESS=DEFLATE",
            "-co", "TILING_SCHEME=GoogleMapsCompatible"
        ]

        # Kjør gdal_translate (cmd) for å konvertere TIFF til COG
        try:
            subprocess.run(cmd, check=True)
            # Tell antall behandla filer
            nTellerAntBehandla += 1
        except subprocess.CalledProcessError as e:
            print(f"Konvertering misslykkes for {tif_file.name}: {e}")
    
    # Sjekk om alle .tif-filer er behandla
    if nTellerAntTiffFiler == 0:
        print("Input-mappa er tom!")
    # Sjekk om alle .tif-filer er behandla
    if nTellerAntBehandla == nTellerAntTiffFiler:
        print("Alle filer konvertert.")6

## Les og legg filene til i STAC-katalogen

In [14]:
convert_folder_to_cog(inputFolder, COGFolder)

nTellerAntCogFiler = 0

for filename in os.listdir(COGFolder):
    if filename.endswith(".tif"): # Velg kun TIFF-filer (COG-filer har også .tif som endelse)
        filepath = COGFolder / filename

        # Les bbox og EPSG-kode fra TIFF-filen
        with rasterio.open(filepath) as src:
            # Hent bounding box for fila
            bounds = src.bounds
            bbox = [bounds.left, bounds.bottom, bounds.right, bounds.top]
            geometry = mapping(box(*bbox)) # Omriss
            crs = src.crs.to_string() # EPSG-kode

        item_id = os.path.splitext(filename)[0]
        item = Item(
            id=item_id,
            geometry=geometry,
            bbox=bbox,
            datetime=datetime.now(),
            properties={"proj:epsg": int(crs.split(":")[-1])},
        )

        # Output-katalog
        item_output_dir = outputFolder / "stac_catalog" / item_id
        item_output_dir.mkdir(parents=True, exist_ok=True)
        item_path = item_output_dir / f"{item_id}.json"

        # Relativ sti til bildet
        rel_image_href = os.path.relpath(filepath, start=item_path.parent)
        item.add_asset("image", Asset(href=rel_image_href, media_type="image/tiff"))

        # Hvis tfw-hjelpefiler finnes legges de til som "georef"
        twf_path = inputFolder / f"{item_id}.tfw"
        if twf_path.exists():
            rel_tfw_href = os.path.relpath(twf_path, start=item_path.parent)
            item.add_asset(
                "georef",
                Asset(
                    href=rel_tfw_href,
                    media_type="text/plain",
                    roles=["auxiliary", "georeferencing"],
                ),
            )

        nTellerAntCogFiler += 1

        catalog.add_item(item)

# Sjekk om alle .cog-filer er behandla
if nTellerAntCogFiler == 0:
    print("Input-mappa er tom!")
else:
    # Lagre katalogen
    catalog.normalize_and_save(
    root_href=str(outputFolder / "stac_catalog"),
    catalog_type="SELF_CONTAINED"
)



Konverterer 32_N50raster_2471.tif til COG...


FileNotFoundError: [WinError 2] The system cannot find the file specified